In [1]:
'''
Deepfake Realism, Context, and Audience Trust 

As synthetic media (“deepfakes”) become more prevalent, audiences must judge the 
authenticity of visual content under varying conditions. In this study, each 
participant viewed one of eight photos (see photos below labeled 1-8) and then 
reported their trust in the photo, their confidence in detecting synthetic media, their 
perception of how confident others are in detecting synthetic media, and their 
digital literacy level. Using the dataset “deepfake.csv”, first assess whether the 
sample appears biased in terms of digital literacy. Does the sample appear to consist 
of more digitally literate or less digitally literate individuals? Next, evaluate whether 
participants’ self-perceived confidence in detecting synthetic media differs from 
their perception of others’ confidence. Finally, test whether the level of visual 
realism (low vs. high) and the context depicted (i.e., news broadcast, political 
speech, entertainment interview, or selfie) independently and jointly contribute to 
perceived trust. What do these patterns suggest about digital literacy, self-other 
perception, and the visual conditions under which audiences are more likely to trust 
synthetic media? 


Variables in “deepfake.csv” 

 trustscore: 0-10; -1 Refused 
 selfconf: 0-10; -1 Refused 
 otherconf: 0-10; -1 Refused 
 digilit: 0-10; 998 Don’t know; 999 Refused; -1 Not applicable 
 PHOTOID: see photos below labeled 1-8 
'''

'\nDeepfake Realism, Context, and Audience Trust \n\nAs synthetic media (“deepfakes”) become more prevalent, audiences must judge the \nauthenticity of visual content under varying conditions. In this study, each \nparticipant viewed one of eight photos (see photos below labeled 1-8) and then \nreported their trust in the photo, their confidence in detecting synthetic media, their \nperception of how confident others are in detecting synthetic media, and their \ndigital literacy level. Using the dataset “deepfake.csv”, first assess whether the \nsample appears biased in terms of digital literacy. Does the sample appear to consist \nof more digitally literate or less digitally literate individuals? Next, evaluate whether \nparticipants’ self-perceived confidence in detecting synthetic media differs from \ntheir perception of others’ confidence. Finally, test whether the level of visual \nrealism (low vs. high) and the context depicted (i.e., news broadcast, political \nspeech, entertain

In [2]:
'''
The given variables (trustscore, selfconf, otherconf, digilit) are all measured on continuous scales (0~10), 
and the independent variables (Realism and Context) are categorical variables, 
so T-test and Analysis of Variance (ANOVA) methodologies should be primarily applied.
'''

'\nThe given variables (trustscore, selfconf, otherconf, digilit) are all measured on continuous scales (0~10), \nand the independent variables (Realism and Context) are categorical variables, \nso T-test and Analysis of Variance (ANOVA) methodologies should be primarily applied.\n'

### 0. Load Data

In [3]:
import pandas as pd

df = pd.read_csv('deepfake.csv')
df.head()

,PHOTOID,realism,context,digilit,selfconf,otherconf,trustscore
0,6,2,2,7,6,4,6
1,2,1,2,6,5,5,4
2,8,2,4,8,8,8,6
3,5,2,1,5,4,5,6
4,7,2,3,998,4,3,6


### 1. Data Preprocessing

In [4]:
# 1.1 Handle missing values and create independent variables
import numpy as np

# Remove missing values (-1, 998, 999)
df_clean = df.copy()
scale_vars = ['trustscore', 'selfconf', 'otherconf', 'digilit']
for var in scale_vars:
    df_clean[var] = df_clean[var].replace([-1, 998, 999], np.nan)
df_clean = df_clean.dropna(subset=scale_vars)

# Create independent variables
df_clean['Visual_Realism'] = df_clean['realism'].map({1: 'Low', 2: 'High'})
df_clean['Context'] = df_clean['context'].map({
    1: 'News', 2: 'Political speech', 
    3: 'Entertainment interview', 4: 'Selfie'
})

print(f"Original: {len(df)}, After preprocessing: {len(df_clean)}")
df_clean.head()

Original: 1000, After preprocessing: 913


,PHOTOID,realism,context,digilit,selfconf,otherconf,trustscore,Visual_Realism,Context
0,6,2,2,7.0,6.0,4.0,6.0,High,Political speech
1,2,1,2,6.0,5.0,5.0,4.0,Low,Political speech
2,8,2,4,8.0,8.0,8.0,6.0,High,Selfie
3,5,2,1,5.0,4.0,5.0,6.0,High,News
5,1,1,1,7.0,8.0,6.0,5.0,Low,News


### 2. Digital Literacy Bias Assessment (One-Sample T-test)

In [5]:
import pandas as pd
from scipy import stats
import numpy as np
import statsmodels.stats.api as sms

# 기술통계
print("=== Descriptive Statistics ===")
print(df_clean['digilit'].describe())

# 디지털 리터러시 데이터 추출
digilit_data = df_clean['digilit']


=== Descriptive Statistics ===
count    913.000000
mean       5.096386
std        1.507844
min        0.000000
25%        4.000000
50%        5.000000
75%        6.000000
max       10.000000
Name: digilit, dtype: float64


In [6]:

'''
Calculate 95% confidence interval to analyze whether the mean includes the neutral point (5)
Interpretation of results:
If confidence interval does not include 5 → Mean is statistically significantly different from 5
If confidence interval includes 5 → Mean does not differ significantly from 5
'''
# Calculate 95% confidence interval for the mean
desc_stats = sms.DescrStatsW(digilit_data)
conf_int = desc_stats.tconfint_mean(alpha=0.05)
mean_digilit = digilit_data.mean()
std_digilit = digilit_data.std()
n = len(digilit_data)

print(f"\nMean: {mean_digilit:.2f}")
print(f"Standard Deviation: {std_digilit:.2f}")
print(f"Sample Size: {n}")
print(f"95% Confidence Interval: ({conf_int[0]:.2f}, {conf_int[1]:.2f})")

if 5 < conf_int[0] or 5 > conf_int[1]:
    print(f"→ Since the confidence interval does not include 5, the mean is statistically significantly different from the neutral point (5)")
else:
    print(f"→ Since the confidence interval includes 5, the mean does not differ significantly from the neutral point (5)")

'''
Perform one-sample t-test
stats.ttest_1samp(): One-sample t-test function from scipy library
popmean=5: Population mean to test (neutral point)

t-statistic: How far the sample mean is from the population mean (5)
p-value: Determines whether the null hypothesis (mean equals 5) can be rejected
p-value < 0.05 → There is a statistically significant difference
'''
# One-sample T-test (H0: μ = 5)
one_sample_ttest = stats.ttest_1samp(digilit_data, popmean=5)
t_stat = one_sample_ttest.statistic
p_value = one_sample_ttest.pvalue

print(f"\n=== One-Sample t-test ===")
print(f"t({n-1}) = {t_stat:.3f}")
print(f"p-value = {p_value:.6f}")

if p_value < 0.001:
    print(f"p < .001 → Strongly reject null hypothesis")
elif p_value < 0.05:
    print(f"p < .05 → Reject null hypothesis")
else:
    print(f"p >= .05 → Accept null hypothesis")

print(f"\nConclusion: Mean ({mean_digilit:.2f}) is {'higher' if mean_digilit > 5 else 'lower'} than the neutral point (5)")
print(f"→ Sample is biased toward individuals with {'high' if mean_digilit > 5 else 'low'} digital literacy")


Mean: 5.10
Standard Deviation: 1.51
Sample Size: 913
95% Confidence Interval: (5.00, 5.19)
→ Since the confidence interval includes 5, the mean does not differ significantly from the neutral point (5)

=== One-Sample t-test ===
t(912) = 1.931
p-value = 0.053733
p >= .05 → Accept null hypothesis

Conclusion: Mean (5.10) is higher than the neutral point (5)
→ Sample is biased toward individuals with high digital literacy


### 3. Comparison of Self-Perception vs Others-Perception

In [7]:
# Paired sample T-test: selfconf vs otherconf
diff = df_clean['selfconf'] - df_clean['otherconf']
mean_diff = diff.mean()
std_diff = diff.std()

t_stat, p_value = stats.ttest_rel(df_clean['selfconf'], df_clean['otherconf'])

print(f"selfconf: M={df_clean['selfconf'].mean():.2f}, SD={df_clean['selfconf'].std():.2f}")
print(f"otherconf: M={df_clean['otherconf'].mean():.2f}, SD={df_clean['otherconf'].std():.2f}")
print(f"Difference: M={mean_diff:.2f}, SD={std_diff:.2f}")
print(f'\nt_stat:{t_stat}\nP_value:{p_value}')
print(f"\nt({len(diff)-1}) = {t_stat:.3f}, p < .001" if p_value < 0.001 else f"\nt({len(diff)-1}) = {t_stat:.3f}, p = {p_value:.3f}")
print(f"\nConclusion: Self-perception is {'higher' if mean_diff > 0 else 'lower'} than others-perception (difference={mean_diff:.2f})")

selfconf: M=5.94, SD=1.66
otherconf: M=5.30, SD=1.92
Difference: M=0.64, SD=1.02

t_stat:18.967869085294943
P_value:6.927622491921137e-68

t(912) = 18.968, p < .001

Conclusion: Self-perception is higher than others-perception (difference=0.64)


### 4. Analysis of Realism and Context Effects on Trust

In [8]:
# Two-Way ANOVA
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Fit model
model = ols('trustscore ~ C(Visual_Realism) * C(Context)', data=df_clean).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

print("Two-Way ANOVA Results:")
print(anova_table.round(3))

# Interpret results
print("\nInterpretation:")
for effect in ['C(Visual_Realism)', 'C(Context)', 'C(Visual_Realism):C(Context)']:
    p = anova_table.loc[effect, 'PR(>F)']
    sig = "Significant (p < .05)" if p < 0.05 else "Not significant"
    effect_name = effect.replace('C(', '').replace(')', '').replace(':', ' × ')
    print(f"- {effect_name}: {sig}")

Two-Way ANOVA Results:
                               sum_sq     df        F  PR(>F)
C(Visual_Realism)             254.420    1.0  648.600     0.0
C(Context)                     41.086    3.0   34.914     0.0
C(Visual_Realism):C(Context)   35.189    3.0   29.903     0.0
Residual                      354.995  905.0      NaN     NaN

Interpretation:
- Visual_Realism: Significant (p < .05)
- Context: Significant (p < .05)
- Visual_Realism × Context: Significant (p < .05)


In [9]:
# Post-hoc analysis: Tukey's HSD if Context main effect is significant
from statsmodels.stats.multicomp import pairwise_tukeyhsd

tukey = pairwise_tukeyhsd(df_clean['trustscore'], df_clean['Context'], alpha=0.05)
print("Tukey's HSD Post-hoc Analysis (Context):")
print(tukey.summary())

Tukey's HSD Post-hoc Analysis (Context):
              Multiple Comparison of Means - Tukey HSD, FWER=0.05              
         group1              group2      meandiff p-adj   lower   upper  reject
-------------------------------------------------------------------------------
Entertainment interview             News   0.1372 0.3029 -0.0656    0.34  False
Entertainment interview Political speech   -0.271 0.0036 -0.4747 -0.0673   True
Entertainment interview           Selfie  -0.4333    0.0 -0.6369 -0.2296   True
                   News Political speech  -0.4082    0.0 -0.6103 -0.2061   True
                   News           Selfie  -0.5705    0.0 -0.7726 -0.3684   True
       Political speech           Selfie  -0.1623  0.168 -0.3653  0.0407  False
-------------------------------------------------------------------------------
